In [1]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from data_extraction.dummy_data_extractor import extract_dummy_data
from statsforecast import StatsForecast

In [2]:
master_table = extract_dummy_data("dummy_data")

row = master_table.iloc[1]
df = row["his"]
df.set_index(df.columns[0], inplace=True, drop=True)
length_of_missing_data = row["dqDuration"]
data_logging_interval = row["pointInterval"]
dqStart	= row['dqStart']
dqDuration = row['dqDuration']

C:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\Sana\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-03-12 01:05:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")


In [14]:
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_squared_error
import numpy as np
from data_extraction.dummy_data_extractor import extract_dummy_data
from statsforecast import StatsForecast

master_table = extract_dummy_data("dummy_data")

row = master_table.iloc[1]
df = row["his"]
df.set_index(df.columns[0], inplace=True, drop=True)
length_of_missing_data = row["dqDuration"]
data_logging_interval = row["pointInterval"]
dqStart	= row['dqStart']
dqDuration = row['dqDuration']


def fProphet(df, length_of_missing_data, data_logging_interval, dqStart):
    """
    Inputs
    df: df used for training set (from SS)
    length_of_missing_data: interval length of missing data (from SS)
    data_logging_interval: data logging interval - called from the hisDQInterval tag on the point (from SS)

    Output
    forecasts_df: dataframe with predictions for the period missing data. Index names as ts, values column named as "v0"
    """

    # Rename the first column as "target"
    new_column_name = "target"
    df = df.rename(columns={df.columns[0]: new_column_name})

    # Keep only the history BEFORE the start of the data quality issue, since this is a statistical model not ML model
    df = df[df.index < dqStart]

    # Format the df to Prophet format
    df = df.reset_index()
    df = df.rename(columns={df.columns[0]: 'ds', df.columns[1]: "y"})

    # Number of predictions
    horizon = int(length_of_missing_data / data_logging_interval) + 1

    # Frequency (Prophet will infer this from the data, but can be specified)
    freq = f'{data_logging_interval.total_seconds() / 3600}H'

    # Initialize and fit the Prophet model
    model = Prophet()
    model.fit(df)

    # Make a dataframe to hold predictions
    future = model.make_future_dataframe(periods=horizon, freq=freq)

    # Make predictions
    forecast = model.predict(future)

    # Filter out the predictions corresponding to the missing data period
    forecasts_df = forecast[['ds', 'yhat']].tail(horizon)
    forecasts_df = forecasts_df.rename(columns={"ds": "timestamp", "yhat": "dynamicOptimizedTheta"})
    forecasts_df.set_index("timestamp", inplace=True)

    return forecasts_df

In [20]:
len(df.columns)

1